# Deep Learning for Autonomous Vehicles
**Module 2: Sub-project 2: NEXET Vehicle Prediction - CSV Processing**
* Student: Dan Sullivan, Spr 2019

## Define Libraries and Functions

In [166]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd
import shutil
import os
import requests
import base64

## Define Paths for Annotation CSVs and YAML Files
Information will be used to create the CSVs

In [167]:
gdrive_csv = 'E:/OneDrive/Documents/School/WPI/RBE595_196E - AI for Autonomous Vehicles/Module 2/Final Project Module 2/nexet/uploaded.csv'
train_boxes_csv = 'E:/OneDrive/Documents/School/WPI/RBE595_196E - AI for Autonomous Vehicles/Module 2/Final Project Module 2/nexet/train_boxes.csv'

## Build Dataframe for Uploaded Images
* Only uploading a fraction of the thousands of images to Gdrive
* Processing the train_boxes csv to only contain information for the uploaded information
* Used the Chrome App, "File Cabinet" to generate a file list of the images uploaded to the drive

In [168]:
df_uploaded = pd.read_csv(gdrive_csv)
df_uploaded.head()

,Title
0,000196c0-1985-457f-bcd2-e4df621de5fe.mov-0001.jpg
1,0012c298-4ec3-4604-a8f5-b828f1775ce6.mov-0001.jpg
2,00146c1b-7af9-4864-9066-9b32913a1c3f.mov-0001.jpg
3,001b630b-1e07-447a-9706-6c0f7aa3c811.mov-0001.jpg
4,0051904c-fc97-43ff-b9c9-14c009c1f6fa.mov-0001.jpg


In [169]:
df_train_orig = pd.read_csv(train_boxes_csv)
df_train_orig.head()



,image_filename,x0,y0,x1,y1,label,confidence
0,frame_817c47b8-22c4-438a-8dc6-0e3f67f299ee_000...,601.600000,270.355731,726.755556,421.185771,van,1.0
1,frame_817c47b8-22c4-438a-8dc6-0e3f67f299ee_000...,497.777778,308.774704,534.755556,338.656126,car,1.0
2,frame_817c47b8-22c4-438a-8dc6-0e3f67f299ee_000...,449.422222,310.197628,509.155556,358.577075,car,1.0
3,frame_a9110bf2-5252-4ec6-83c6-33b65d0fc04d_000...,711.111111,304.505929,786.488889,368.537549,car,1.0
4,frame_a9110bf2-5252-4ec6-83c6-33b65d0fc04d_000...,584.533333,307.351779,647.111111,358.577075,car,1.0


## Build New DF For Colab Processing

In [170]:
# Sort both dataframes by the image name for faster processing
df_uploaded.sort_values(by=['Title'],inplace=True)
df_uploaded = df_uploaded.reset_index(drop=True)
df_uploaded.rename(index=str,columns={'Title':'image_filename'},inplace=True)
print("df_uploaded:")
print(df_uploaded[0:5],"\n")

df_train_orig.sort_values(by=['image_filename'],inplace=True)
df_train_orig = df_train_orig.reset_index(drop=True)

print("df_train_orig:")
print(df_train_orig[0:5],"\n")

# Initialize new dataframe for Colab Processing
df_train_colab = pd.DataFrame
df_train_colab = pd.merge(df_uploaded,
                          df_train_orig[['image_filename','x0','x1','y0','y1','label']],
                          on='image_filename')

print("df_train_colab:")
print(df_train_colab[0:5],"\n")


df_uploaded:
                                      image_filename
0  000196c0-1985-457f-bcd2-e4df621de5fe.mov-0001.jpg
1  0012c298-4ec3-4604-a8f5-b828f1775ce6.mov-0001.jpg
2  00146c1b-7af9-4864-9066-9b32913a1c3f.mov-0001.jpg
3  001b630b-1e07-447a-9706-6c0f7aa3c811.mov-0001.jpg
4  0051904c-fc97-43ff-b9c9-14c009c1f6fa.mov-0001.jpg 

df_train_orig:
                                      image_filename          x0          y0  \
0  0012c298-4ec3-4604-a8f5-b828f1775ce6.mov-0001.jpg  603.022222  288.853755   
1  0012c298-4ec3-4604-a8f5-b828f1775ce6.mov-0001.jpg  496.355556  330.118577   
2  0012c298-4ec3-4604-a8f5-b828f1775ce6.mov-0001.jpg  321.422222  305.928854   
3  0012c298-4ec3-4604-a8f5-b828f1775ce6.mov-0001.jpg    0.000000  283.162055   
4  0012c298-4ec3-4604-a8f5-b828f1775ce6.mov-0001.jpg  614.400000  298.814229   

           x1          y1         label  confidence  
0  649.955556  344.347826         truck         1.0  
1  551.822222  360.000000  pickup_truck         1.0  
2  452.26

In [171]:
print(len(df_uploaded['image_filename']))
print(len(df_train_orig['image_filename']))
print(len(df_train_colab['image_filename']))

3819
134361
9868


In [172]:
out_csv = 'E:/OneDrive/Documents/School/WPI/RBE595_196E - AI for Autonomous Vehicles/Module 2/Final Project Module 2/nexet/output.csv'
df_train_colab.sort_values(by=['image_filename'],inplace=True)
df_train_colab = df_train_colab.reset_index(drop=True)

colab_train_img_base_dir = '/content/gdrive/My Drive/AI/NEXET/nexet_2017_train/images/'  

# Add Gdrive path to each filename string
df_train_colab['image_filename'] = colab_train_img_base_dir + df_train_colab['image_filename'].astype(str)

# Round the x/y min, max values to integers (pixels)
df_train_colab = df_train_colab.round({'x0': 0, 'x1': 1, 'y0': 0, 'y1': 0})
df_train_colab['x0'] = df_train_colab['x0'].astype(int)
df_train_colab['x1'] = df_train_colab['x1'].astype(int)
df_train_colab['y0'] = df_train_colab['y0'].astype(int)
df_train_colab['y1'] = df_train_colab['y1'].astype(int)


df_train_colab.to_csv(out_csv, index=False, header=False)